In [1]:
from pymongo import MongoClient

In [ ]:
class TwitterSearchApp:
    def __init__(self, max_cache_size=100):
        self.max_cache_size = max_cache_size
        self.cache = {}
        self.client = MongoClient('mongodb://localhost:27017/')
        self.cache_collection = self.client['twitter_db']['cache']

    def load_cache_from_mongodb(self):
        cache_data = self.cache_collection.find_one()
        if cache_data:
            self.cache = cache_data['cache']

    def search_tweets(self, keyword):
        if keyword in self.cache:
            print("Result found in cache:")
            return self.cache[keyword]
        else:
            print("Searching in MongoDB:")
            # Placeholder for MongoDB query, replace with actual query to search for tweets
            result = self.query_mongodb(keyword)
            if len(self.cache) < self.max_cache_size:
                self.cache[keyword] = result
                self.cache_collection.update_one({}, {'$set': {'cache': self.cache}}, upsert=True)
            return result

    def query_mongodb(self, keyword):
        # Placeholder for MongoDB query, replace with actual query to search for tweets
        return f"Dummy result for {keyword}"

    def shutdown(self):
        self.cache_collection.update_one({}, {'$set': {'cache': self.cache}}, upsert=True)
        self.client.close()

# Example usage
if __name__ == "__main__":
    app = TwitterSearchApp(max_cache_size=10)
    app.load_cache_from_mongodb()

    # Example searches
    print(app.search_tweets("python"))
    print(app.search_tweets("mongodb"))
    print(app.search_tweets("data science"))

    app.shutdown()

In [11]:
import nltk
# nltk.download('punkt')
# nltk.download('stopwords')

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/mushirashaikh/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/mushirashaikh/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [1]:
from app import mongo_db_connection
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from collections import Counter


In [2]:
# Connect to MongoDB and fetch the collection
mongo_client = mongo_db_connection()
mongo_db = mongo_client.sample
mongo_collection = mongo_db.tweets

Connected to MongoDB!


In [7]:
def get_top_10_keywords():
# parse through each tweet and maintain a dict of top 10 keywords
    # Retrieve all tweets
    tweets = mongo_collection.find({}, {"text": 1})
    # print(len(list(tweets)))
    
    # Define stopwords - words to ignore
    stop_words = set(stopwords.words('english'))
    

    # Count occurrences of each keyword in all tweets
    keyword_count = Counter()
    for tweet in tweets:
        text = tweet["text"]
        keywords = extract_keywords(text, stop_words)
        keyword_count.update(keywords)

    # Get top 10 keywords
    return keyword_count.most_common(10)


print(get_top_10_keywords())

[('https', 13048), ('corona', 10474), ('virus', 2248), ('de', 1718), ('die', 1105), ('que', 831), ('la', 777), ('bu', 702), ('en', 655), ('der', 625)]


In [6]:
# Define function to process text and extract keywords
def extract_keywords(text, stop_words):
    tokens = word_tokenize(text.lower())  # Tokenize text
    filtered_tokens = [word for word in tokens if word.isalnum() and word not in stop_words]  # Remove stopwords and non-alphanumeric tokens
    return filtered_tokens